<h1 style='font-weight:bold; text-align: justify'> Test </h1>
<h1 style='font-weight:bold; text-align: justify'> Machine Learning STQD6024 </h1>
<h1 style='font-weight:bold; text-align: justify'> Hazim Fitri Bin Ahmad Faudzi P152419 </h1>

<h1 style='font-weight:bold; color:orange;'> a) Model building </h1>

In [42]:
# import necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import time
import itertools

In [43]:
# read data
df = pd.read_table('Question_2.txt', sep=',')
df

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
95,96,184.3,153.4,239.2,179.0,7.5,920.2,640.9,0.0,20.0,41.01
96,97,215.6,112.9,239.0,198.7,7.4,884.0,649.1,27.5,64.0,39.13
97,98,295.3,0.0,239.9,236.2,8.3,780.3,722.9,25.0,77.0,44.08
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97


In [44]:
# define response (Y) and predictor (X) variables
y = df['Compressive Strength (28-day)(Mpa)']
x = df.drop(['Compressive Strength (28-day)(Mpa)', 'SLUMP(cm)', 'FLOW(cm)', 'No'], axis = 1)

In [45]:
# split data intro training set and testing set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [46]:
# check variables data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   No                                  100 non-null    int64  
 1   Cement                              100 non-null    float64
 2   Slag                                100 non-null    float64
 3   Fly ash                             100 non-null    float64
 4   Water                               100 non-null    float64
 5   SP                                  100 non-null    float64
 6   Coarse Aggr.                        100 non-null    float64
 7   Fine Aggr.                          100 non-null    float64
 8   SLUMP(cm)                           100 non-null    float64
 9   FLOW(cm)                            100 non-null    float64
 10  Compressive Strength (28-day)(Mpa)  100 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 8.7 KB


In [47]:
scale = StandardScaler()

x_train_scale = scale.fit_transform(x_train)
x_test_scale = scale.fit_transform(x_test)

In [48]:
lr = LinearRegression()

model = lr.fit(x_train_scale, y_train)

y_predict = model.predict(x_train_scale)

y_predict_test = model.predict(x_test_scale)

col_name = x.columns

print("Regression model coefficients:")
print(pd.DataFrame( {'Variables':col_name, 'Coefficient':lr.coef_}))
print('============================\n')
print(f"Regression model intercept:{lr.intercept_:.4f}")

Regression model coefficients:
      Variables  Coefficient
0        Cement     4.898330
1          Slag    -1.755212
2       Fly ash     4.658713
3         Water    -4.545214
4            SP     0.423314
5  Coarse Aggr.    -4.638413
6    Fine Aggr.    -2.257704

Regression model intercept:35.8230


<h1 style='font-weight:bold; color:orange;'> b) Compute MSE on test set </h1>

In [49]:
print(f"MSE for training set: {mean_squared_error(y_train, y_predict):.4f}")
print(f"MSE for testing set: {mean_squared_error(y_test, y_predict_test):.4f}")

MSE for training set: 6.8980
MSE for testing set: 6.9671


<h1 style='font-weight:bold; color:orange;'> c) Optimal model using forward selection </h1>

In [63]:

def processSubset(feature_set):

    model = sm.OLS(y,x[list(feature_set)].astype(float))

    regr = model.fit()

    RSS = ((regr.predict(x[list(feature_set)]) - y) ** 2).sum()

    return {"model":regr, "RSS":RSS}

def forward(predictors):

    remaining_predictors = [p for p in x.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:

        results.append(processSubset(predictors+[p]))
    
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()

    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
   
    return best_model

models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()

predictors = []

for i in range(1,len(x.columns)+1):    

    models_fwd.loc[i] = forward(predictors)
    
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  7 models on 1 predictors in 0.0065495967864990234 seconds.
Processed  6 models on 2 predictors in 0.0 seconds.
Processed  5 models on 3 predictors in 0.0 seconds.
Processed  4 models on 4 predictors in 0.0 seconds.
Processed  3 models on 5 predictors in 0.013337135314941406 seconds.
Processed  2 models on 6 predictors in 0.0013492107391357422 seconds.
Processed  1 models on 7 predictors in 0.0 seconds.
Total elapsed time: 0.022236108779907227 seconds.


In [62]:
print(models_fwd.loc[6, "model"].summary())

                                         OLS Regression Results                                        
Dep. Variable:     Compressive Strength (28-day)(Mpa)   R-squared (uncentered):                   0.995
Model:                                            OLS   Adj. R-squared (uncentered):              0.995
Method:                                 Least Squares   F-statistic:                              3143.
Date:                                Sat, 21 Jun 2025   Prob (F-statistic):                   5.66e-106
Time:                                        10:56:32   Log-Likelihood:                         -236.11
No. Observations:                                 100   AIC:                                      484.2
Df Residuals:                                      94   BIC:                                      499.8
Df Model:                                           6                                                  
Covariance Type:                            nonrobust           

From the forward selection, we can see that the best model contain 6 variables as indicates by highest adjusted r-squared of 0.995. Increasing the number of variables will not increase the model performance.

<h1 style='font-weight:bold; color:orange;'> d) Model prediction </h1>

In [ ]:
x_new = ['Fine Aggr.', 'Fly ash', 'Cement', 'Water', 'Slag', 'SP']

df.drop(['Compressive Strength (28-day)(Mpa)', 'SLUMP(cm)', 'FLOW(cm)', 'No', Coarse Aggr.])

In [51]:
new_df = pd.DataFrame({'Cement':[258.8,348.7], 'Slag':[88,0.1], 'Fly Ash':[239.6, 223.1], 'Water':[175.3, 208.5], 'SP':[7.6, 9.6], 'Coarse Aggr.':[938.9, 786.2], 'Fine Aggr.':[646, 758.1]})
new_df

,Cement,Slag,Fly Ash,Water,SP,Coarse Aggr.,Fine Aggr.
0,258.8,88.0,239.6,175.3,7.6,938.9,646.0
1,348.7,0.1,223.1,208.5,9.6,786.2,758.1


In [52]:
model.predict(new_df)

c:\Users\PC 13\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([-4341.76193057, -3518.84509646])